In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_dir = "/content/drive/My Drive/Polygence/Audrey/"

In [0]:
import urllib
from bs4 import BeautifulSoup
import os
import json
from urllib.error import HTTPError
import re

## Load sources & candidates

In [0]:
candidates = open(root_dir + "data/candidate_names.txt", "r").read().splitlines() + ["Michael Bloomberg"]
sources = open(root_dir + "data/source_names.txt", "r").read().splitlines()

In [0]:
sources

['abc-news',
 'al-jazeera-english',
 'associated-press',
 'axios',
 'breitbart-news',
 'cbs-news',
 'cnn',
 'fox-news',
 'google-news',
 'msnbc',
 'national-review',
 'nbc-news',
 'newsweek',
 'new-york-magazine',
 'politico',
 'reddit-r-all',
 'reuters',
 'the-american-conservative',
 'the-hill',
 'the-huffington-post',
 'the-new-york-times',
 'the-washington-post',
 'the-washington-times',
 'time',
 'usa-today',
 'vice-news']

In [0]:
candidates

['Michael Bennet',
 'Joe Biden',
 'Bill de Blasio',
 'Cory Booker',
 'Steve Bullock',
 'Pete Buttigieg',
 'Julián Castro',
 'John Delaney',
 'Tulsi Gabbard',
 'Kirsten Gillibrand',
 'Kamala Harris',
 'Jay Inslee',
 'Amy Klobuchar',
 'Wayne Messam',
 'Seth Moulton',
 "Beto O'Rourke",
 'Tim Ryan',
 'Bernie Sanders',
 'Joe Sestak',
 'Tom Steyer',
 'Elizabeth Warren',
 'Marianne Williamson',
 'Andrew Yang',
 'Michael Bloomberg']

## Inspect what a particular domain's html structure looks like

In [0]:
example_candidate = "Andrew Yang"
source = "cnn"

### Print & copy an example headline + metadata from this source

In [0]:
with open(root_dir + "data/headlines/%s_%s.json" % (example_candidate, source), "r") as f:
  articles = json.load(f)
  for a in articles["articles"]:
      if a["source"]["id"] == source:
        
        url = a["url"]
        if "videos/" in url:
          continue
        title = a["title"]
        content = a["content"]
        print(a)
        break

{'source': {'id': 'cnn', 'name': 'CNN'}, 'author': 'Photographs by Mark Peterson/Redux for CNN', 'title': 'Politics and corn dogs at Iowa State Fair', 'description': 'The Iowa State Fair has become an almost-mandatory stop for candidates looking to gain an edge in the presidential race.', 'url': 'https://www.cnn.com/2019/08/09/politics/gallery/iowa-state-fair-2019/index.html', 'urlToImage': 'https://cdn.cnn.com/cnnnext/dam/assets/190809190644-01-iowa-state-fair-super-tease.jpg', 'publishedAt': '2019-08-09T23:52:53Z', 'content': 'Democratic presidential candidate Andrew Yang eats a corn dog at the Iowa State Fair on Friday, August 9. The corn dog is just one of 69 food items that you can find on a stick there.\r\nThe Iowa State Fair is a sight to behold. \r\nThere are carnival rides, conc… [+605 chars]'}


In [0]:
title

'Politics and corn dogs at Iowa State Fair'

In [0]:
url

'https://www.cnn.com/2019/08/09/politics/gallery/iowa-state-fair-2019/index.html'

In [0]:
content

'Democratic presidential candidate Andrew Yang eats a corn dog at the Iowa State Fair on Friday, August 9. The corn dog is just one of 69 food items that you can find on a stick there.\r\nThe Iowa State Fair is a sight to behold. \r\nThere are carnival rides, conc… [+605 chars]'

In [0]:
soup = parse_url(url)

In [0]:
art_body = soup.find_all(class_=re.compile("body__paragraph"))
art_body

[<div class="el-unfurled__caption">US Sen. Bernie Sanders and his wife, Jane, walk through the fairgrounds on Saturday. Most of the Democratic field visited the fair over the weekend.<div class="el-unfurled__credit">Mark Peterson/Redux for CNN</div></div>,
 <div class="el-unfurled__caption">A fairgoer casts a vote at the kernel voting station.<div class="el-unfurled__credit">Mark Peterson/Redux for CNN</div></div>,
 <div class="el-unfurled__caption">It wasn't just Democratic candidates attending the fair this year. Bill Weld, a former Massachusetts governor who is challenging President Trump for the Republican nomination, talks to the media on Sunday.<div class="el-unfurled__credit">Mark Peterson/Redux for CNN</div></div>,
 <div class="el-unfurled__caption">Dark clouds loom over the Ferris wheel on Sunday.<div class="el-unfurled__credit">Mark Peterson/Redux for CNN</div></div>,
 <div class="el-unfurled__caption">Fairgoers take shelter under an umbrella during a rainy Sunday.<div class=

In [0]:
for e in art_body:
  print(e.text)

US Sen. Bernie Sanders and his wife, Jane, walk through the fairgrounds on Saturday. Most of the Democratic field visited the fair over the weekend.Mark Peterson/Redux for CNN
A fairgoer casts a vote at the kernel voting station.Mark Peterson/Redux for CNN
It wasn't just Democratic candidates attending the fair this year. Bill Weld, a former Massachusetts governor who is challenging President Trump for the Republican nomination, talks to the media on Sunday.Mark Peterson/Redux for CNN
Dark clouds loom over the Ferris wheel on Sunday.Mark Peterson/Redux for CNN
Fairgoers take shelter under an umbrella during a rainy Sunday.Mark Peterson/Redux for CNN
A young man poses with cutouts of President Trump and first lady Melania Trump.Mark Peterson/Redux for CNN
US Sen. Kamala Harris speaks to fairgoers on Saturday. The Des Moines Register gives each candidate a soapbox, a microphone and 20 minutes to address fair attendees.Mark Peterson/Redux for CNN
More than 100,000 people attended the fair

In [0]:
def inspect_domain(example_url):
    req = urllib.request.Request(example_url, headers={'User-Agent' : "Magic Browser"})
    con = urllib.request.urlopen( req )
    html = con.read()
    soup = BeautifulSoup(html,'html.parser')
    print(soup.prettify())

Follow the steps in [this document](https://docs.google.com/document/d/1KdrgxB07sdcGGy8kqR_-FsEDMtLkSNTOYmo6QLs15IU/edit). If you want to get a better understanding of the html structure, you can also look at the html output below.

In [0]:
inspect_domain(url)

NameError: ignored

## Get full articles

In [0]:
def parse_url(url):
  req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
  try:
    con = urllib.request.urlopen( req )
  except HTTPError:
    print("%s not found" % url)
    return None
  html = con.read()
  return BeautifulSoup(html,'html.parser')

In [0]:
def get_full_article_text(url, domain):
  soup = parse_url(url)
  
  # Filter out non-existent HTMLs
  if not soup:
    return None
 
  if domain == 'the-new-york-times':
    try:
      article = soup.find('article',attrs={'id':'story'})
      body = article.find('section',attrs={'name':'articleBody'})
      paragraphs = body.find_all('p')
    except AttributeError:
      paragraphs = soup.find_all('p',attrs={'class':True})
          
  elif domain == 'fox-news':
    
    # Filter out videos 
    if "video.foxnews" in url:
      return None
    
    art_body = soup.find_all('div',class_=re.compile("article-body|td-post-text-content"))  # a regular expression for searching for either one of the class types (article-body or td-post-text-content)
    if len(art_body) != 1:
      print("article body doesn't match for %s" % url) # check the link to make sure it's indeed because there is no text content
      return None
    paragraphs = art_body[0].find_all('p')
      
  elif domain == 'breitbart-news':
    article = soup.find_all('article')
    if len(article) != 1:
      print("article doesn't match for %s" % url)
      return None
    art_body = article[0].find_all('div',attrs={'class':'entry-content'},recursive=False)
    if len(art_body) != 1:
      print("article body doesn't match for %s" % url)
      return None
    paragraphs = []
    h2 = art_body[0].find('h2')
    if h2 is not None:
      paragraphs.append(h2)
    for ab in art_body:
      paragraphs.extend(ab.find_all('p'))
          
  elif domain == 'national-review':
    paragraphs = soup.find_all('p') # done
    
  elif domain == 'abc-news':
    paragraphs = soup.find_all('p') # done
    
  elif domain == 'al-jazeera-english':
    paragraphs = soup.find_all('p') #done
  
  elif domain == 'associated-press':
    paragraphs = soup.find_all('p') # done

  elif domain == 'axios':
    paragraphs = soup.find_all('p') # done
    
  elif domain == 'cbs-news':  #done
    if "/videos/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
  
  elif domain == 'cnn': ##?? search by class
    if "/videos/" in url:
      return None
    else:
      paragraphs = soup.find_all(class_=re.compile("body__paragraph"))
  
  elif domain == 'google-news':
    paragraphs = soup.find_all('p') #  ????
  
  elif domain == 'msnbc': # done
    if "/watch/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'nbc-news': #done
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'newsweek':
    paragraphs = soup.find_all('p')
   
  elif domain == 'new-york-magazine':
    paragraphs = soup.find_all('p')
    
  elif domain == 'politico':
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'reddit-r-all':
    paragraphs = soup.find_all('p') ###??
    
  elif domain == 'reuters':
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'the-american-conservative':
    paragraphs = soup.find_all('p')
    
  elif domain == 'the-hill':
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'the-huffington-post':
    paragraphs = soup.find_all('p')
    
  elif domain == 'the-washington-post':
    if "/video/" in url:
      return None
    else:
        paragraphs = soup.find_all('p')
    
  elif domain == 'the-washington-times':
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'time':
    paragraphs = soup.find_all('p')
    
  elif domain == 'usa-today':
    if "/video/" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
    
  elif domain == 'vice-news':
    if "video.vice" in url:
      return None
    else:
      paragraphs = soup.find_all('p')
      
  # Add your methods for the new sources here

  return " ".join([p.text for p in paragraphs])

In [0]:
def get_all_articles_for_source(source):
  print("Getting articles for %s..." % source)
  for candidate in candidates:
    print(candidate)
    
    # Open new file for writing (change full articles directory if necessary)
    with open(root_dir + "data/full_articles/%s_%s.json" % (candidate, source), "a+") as fw:
         
      # Open headlines file (change headlines directory if necessary)
      with open(root_dir + "data/headlines_3/%s_%s.json" % (candidate, source), "r") as fr:
        articles = json.load(fr)

        # Loop through articles
        num_articles = 0
        for a in articles["articles"]:
          if a["source"]["id"] == source:
            num_articles +=1

            # Get full text based on the URL and domain
            full_text = get_full_article_text(a["url"], source)
            if not full_text:
              continue
 
            d = {"source": source, "author": a["author"], "title": a["title"],
                "url": a["url"], "publishedAt": a["publishedAt"], "content": full_text}

            # Write to file
            fw.write(json.dumps(d) + "\n")
        print("%d articles saved" % num_articles)
           

In [0]:
for source in sources[9:]:
  if source == "the-new-york-times":
    continue
  get_all_articles_for_source(source)

Getting articles for msnbc...
Michael Bennet
5 articles saved
Joe Biden
36 articles saved
Bill de Blasio
4 articles saved
Cory Booker
19 articles saved
Steve Bullock
0 articles saved
Pete Buttigieg
22 articles saved
Julián Castro
4 articles saved
John Delaney
1 articles saved
Tulsi Gabbard
8 articles saved
Kirsten Gillibrand
0 articles saved
Kamala Harris
1 articles saved
Jay Inslee
0 articles saved
Amy Klobuchar
16 articles saved
Wayne Messam
0 articles saved
Seth Moulton
2 articles saved
Beto O'Rourke
0 articles saved
Tim Ryan
2 articles saved
Bernie Sanders
30 articles saved
Joe Sestak
0 articles saved
Tom Steyer
14 articles saved
Elizabeth Warren
26 articles saved
Marianne Williamson
2 articles saved
Andrew Yang
9 articles saved
Michael Bloomberg
18 articles saved
Getting articles for national-review...
Michael Bennet
1 articles saved
Joe Biden
4 articles saved
Bill de Blasio
0 articles saved
Cory Booker
0 articles saved
Steve Bullock
0 articles saved
Pete Buttigieg
6 articles save

In [0]:
get_all_articles_for_source("the-washington-post")

Getting articles for the-washington-post...
Beto O'Rourke
29 articles saved
Tim Ryan
26 articles saved
Bernie Sanders
100 articles saved
Joe Sestak
6 articles saved
Tom Steyer
34 articles saved
Elizabeth Warren
100 articles saved
Marianne Williamson
16 articles saved
Andrew Yang
